# Assignment 3

# Evaluate

+ Which model has the best performance?

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import shap
import pickle
import warnings
warnings.filterwarnings('ignore')



IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [10]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [12]:
X = fires_dt.drop('area', axis=1)
y = np.log1p(fires_dt['area'])  # Log transform target variable due to skewness

# Define feature types
numeric_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
categorical_features = ['month', 'day']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create log transformation function for features
def log_transform(X):
   
    return np.where(X > 0, np.log1p(X), X)

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [13]:
# Preprocessor 1: Simple scaling
preproc1 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), categorical_features)
    ])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [14]:
preproc2 = ColumnTransformer(
    transformers=[
        ('num_log', Pipeline([
            ('log', FunctionTransformer(log_transform)),
            ('scale', StandardScaler())
        ]), ['dmc', 'dc', 'isi']),  # Apply log transform to heavily skewed features
        ('num_standard', StandardScaler(), ['coord_x', 'coord_y', 'ffmc', 'temp', 'rh', 'wind', 'rain']),
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), categorical_features)
    ])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [15]:
# Pipeline A = preproc1 + baseline
pipeline_a = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', KNeighborsRegressor())
])

In [16]:
# Pipeline B = preproc2 + baseline
pipeline_b = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', KNeighborsRegressor())
])

In [17]:
# Pipeline C = preproc1 + advanced model
pipeline_c = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', RandomForestRegressor(random_state=42))
])


In [18]:
# Pipeline D = preproc2 + advanced model
pipeline_d = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', RandomForestRegressor(random_state=42))
])

    

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [20]:
param_grid_knn = {
    'regressor__n_neighbors': [3, 5, 7, 9],
    'regressor__weights': ['uniform', 'distance']
}

param_grid_rf = {
    'regressor__n_estimators': [100, 200],
    'regressor__max_depth': [10, 20],
    'regressor__min_samples_split': [2, 5]
}

In [22]:
# Perform GridSearch 
models = {
    'Pipeline A (Simple + KNN)': (pipeline_a, param_grid_knn),
    'Pipeline B (Log + KNN)': (pipeline_b, param_grid_knn),
    'Pipeline C (Simple + RF)': (pipeline_c, param_grid_rf),
    'Pipeline D (Log + RF)': (pipeline_d, param_grid_rf)
}

results = {}
for name, (pipeline, param_grid) in models.items():
    print(f"\nFitting {name}...")
    grid_search = GridSearchCV(
        pipeline, 
        param_grid, 
        cv=5, 
        scoring='neg_root_mean_squared_error',
        n_jobs=-1
    )
    grid_search.fit(X_train, y_train)
    
    
    y_pred = grid_search.predict(X_test)
    y_pred_orig = np.expm1(y_pred)
    y_test_orig = np.expm1(y_test)
    

    rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))
    mae = mean_absolute_error(y_test_orig, y_pred_orig)
    
    results[name] = {
        'model': grid_search,
        'RMSE': rmse,
        'MAE': mae,
        'Best Parameters': grid_search.best_params_
    }
    
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"Best Parameters: {grid_search.best_params_}")


Fitting Pipeline A (Simple + KNN)...
RMSE: 110.06
MAE: 19.92
Best Parameters: {'regressor__n_neighbors': 9, 'regressor__weights': 'distance'}

Fitting Pipeline B (Log + KNN)...
RMSE: 110.06
MAE: 19.84
Best Parameters: {'regressor__n_neighbors': 9, 'regressor__weights': 'distance'}

Fitting Pipeline C (Simple + RF)...


/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/minicon

RMSE: 109.96
MAE: 19.86
Best Parameters: {'regressor__max_depth': 10, 'regressor__min_samples_split': 5, 'regressor__n_estimators': 200}

Fitting Pipeline D (Log + RF)...


/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/minicon

RMSE: 109.96
MAE: 19.87
Best Parameters: {'regressor__max_depth': 10, 'regressor__min_samples_split': 5, 'regressor__n_estimators': 200}


In [23]:
best_model_name = min(results.items(), key=lambda x: x[1]['RMSE'])[0]
best_model = results[best_model_name]['model']
print(f"\nBest Model: {best_model_name}")
print(f"Best RMSE: {results[best_model_name]['RMSE']:.2f}")


Best Model: Pipeline C (Simple + RF)
Best RMSE: 109.96


# Export

+ Save the best performing model to a pickle file.

In [ ]:
import pickle

# Save the best performing model to a pickle file
with open('best_pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline_c, file)

with open('best_pipeline.pkl', 'rb') as file:
    load_pipeline = pickle.load(file)
load_pipeline


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['month', 'day'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [39]:
import shap

# Get the best model 
best_pipeline = results['Pipeline C (Simple + RF)']['model'].best_estimator_
rf_model = best_pipeline.named_steps['regressor']


X_processed = best_pipeline.named_steps['preprocessing'].transform(X_test)
sample_idx = 0

# Get original feature values 
original_values = X_test.iloc[sample_idx]
print("\nOriginal feature values for selected observation:")
print("-------------------------------------------------")
print(original_values)

# SHAP explainer 
explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(X_processed[sample_idx:sample_idx+1])

# Print shape information for debugging
print(f"\nShape of SHAP values: {shap_values[0].shape}")
print(f"Number of numeric features: {len(numeric_features)}")
print(f"Number of encoded categorical features: {X_processed.shape[1] - len(numeric_features)}")

# Get feature contributions for this prediction
importances = pd.DataFrame({
    'Feature': numeric_features,
    'Original Value': original_values[numeric_features],
    'SHAP Value': shap_values[0][:len(numeric_features)]
})

# Sort by absolute SHAP value
importances['Abs SHAP Value'] = importances['SHAP Value'].abs()
importances = importances.sort_values('Abs SHAP Value', ascending=False)

print("\nFeature contributions to prediction for this observation:")
print("------------------------------------------------------")
print(importances[['Feature', 'Original Value', 'SHAP Value']])

# Get the prediction for this observation
prediction = best_pipeline.predict(X_test.iloc[[sample_idx]])
print(f"\nPredicted fire area (log scale): {prediction[0]:.3f}")
print(f"Predicted fire area (original scale): {np.expm1(prediction[0]):.3f} ha")




Original feature values for selected observation:
-------------------------------------------------
coord_x        6
coord_y        5
month        may
day          sat
ffmc        85.1
dmc         28.0
dc         113.8
isi          3.5
temp        11.3
rh            94
wind         4.9
rain         0.0
Name: 304, dtype: object

Shape of SHAP values: (27,)
Number of numeric features: 10
Number of encoded categorical features: 17

Feature contributions to prediction for this observation:
------------------------------------------------------
         Feature Original Value  SHAP Value
dmc          dmc           28.0   -0.156867
rh            rh             94   -0.089735
temp        temp           11.3   -0.075336
ffmc        ffmc           85.1    0.059205
coord_x  coord_x              6   -0.032267
isi          isi            3.5   -0.023839
coord_y  coord_y              5   -0.021425
dc            dc          113.8    0.017628
wind        wind            4.9    0.000376
rain        r

In [41]:


# Calculate SHAP values for the entire training set
shap_values = explainer.shap_values(X_train_processed)

# Calculate mean absolute SHAP values for each feature
mean_abs_shap = np.abs(shap_values).mean(axis=0)

# Create a dataframe of feature importances
feature_importance = pd.DataFrame({
    'Feature': numeric_features,
    'Mean |SHAP Value|': mean_abs_shap[:len(numeric_features)]
})

# Sort by importance
feature_importance = feature_importance.sort_values('Mean |SHAP Value|', ascending=False)

print("\nGlobal Feature Importance Rankings:")
print("---------------------------------")
print(feature_importance)

print("\nMost Important Features:")
print("----------------------")
print(feature_importance.head())

print("\nLeast Important Features:")
print("----------------------")
print(feature_importance.tail())


Global Feature Importance Rankings:
---------------------------------
   Feature  Mean |SHAP Value|
3      dmc           0.145459
6     temp           0.139320
0  coord_x           0.102683
7       rh           0.080312
4       dc           0.065482
5      isi           0.057697
2     ffmc           0.050403
8     wind           0.047343
1  coord_y           0.026610
9     rain           0.000514

Most Important Features:
----------------------
   Feature  Mean |SHAP Value|
3      dmc           0.145459
6     temp           0.139320
0  coord_x           0.102683
7       rh           0.080312
4       dc           0.065482

Least Important Features:
----------------------
   Feature  Mean |SHAP Value|
5      isi           0.057697
2     ffmc           0.050403
8     wind           0.047343
1  coord_y           0.026610
9     rain           0.000514


Feature to remove:

Rain:  Low importance and likely redundant with other moisture-related variables.
The information might be better captured by FFMC and humidity measurements




*(Answer here.)*

## Criteria

The [rubric](./assignment_3_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.